In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.optimize import curve_fit, OptimizeWarning
from tqdm import tqdm
import warnings
from scipy.stats import zscore
from statsmodels.tsa.stattools import acf, pacf
from scipy.optimize import minimize


sns.set_theme()
sns.set_context("notebook")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
dtype_dict = {
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'AnimalNumber': 'Int64',          
    'StartDate': 'str',
    'StartTime': 'str',
    'DateTime': 'str',
    'LactationNumber': 'Int64',       
    'DaysInMilk': 'Int64', 
    'YearSeason': 'str',           
    'TotalYield': 'float',
    'DateTime': 'str',
    'BreedName': 'str',
    'Age': 'Int64',
    'Mother': 'str',
    'Father': 'str',
    'CullDecisionDate': 'str',
    'Temperature': 'float',
    'RelativeHumidity': 'float',      
    'THI_adj': 'float',
    'HW': 'Int64',                    
    'cum_HW': 'Int64',                
    'Temp15Threshold': 'Int64'        
}


# Load the CSV with specified dtypes
data = pd.read_csv('../Data/MergedData/CleanedYieldData.csv', dtype=dtype_dict)

# Convert date and time columns back to datetime and time objects
data['DateTime'] = pd.to_datetime(data['DateTime'], errors='coerce')
data['StartTime'] = pd.to_datetime(data['StartTime'], format='%H:%M:%S', errors='coerce').dt.time
data['StartDate'] = pd.to_datetime(data['StartDate'], errors='coerce')
data['CullDecisionDate'] = pd.to_datetime(data['CullDecisionDate'], errors='coerce')
data['DateTime'] = pd.to_datetime(data['DateTime'], errors='coerce')
data.head()

,FarmName_Pseudo,SE_Number,AnimalNumber,StartDate,StartTime,LactationNumber,DaysInMilk,TotalYield,DateTime,YearSeason,...,Mother,Father,CullDecisionDate,Temperature,RelativeHumidity,THI_adj,HW,cum_HW,Temp15Threshold,Age
0,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,06:25:00,7,191,13.90,2022-01-01 06:25:00,2022-1,...,NaN,NaN,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095
1,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,16:41:00,7,191,16.87,2022-01-01 16:41:00,2022-1,...,NaN,NaN,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095
2,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,15:29:00,7,192,20.41,2022-01-02 15:29:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096
3,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,03:31:00,7,192,16.28,2022-01-02 03:31:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096
4,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,22:44:00,7,192,11.53,2022-01-02 22:44:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096


In [10]:
# Define the THI threshold
THI_THRESHOLD = 61

# Calculate the daily heat load based on the THI threshold
data['HeatLoad'] = data['THI_adj'].apply(lambda x: x - THI_THRESHOLD if x > THI_THRESHOLD else -(THI_THRESHOLD - x))

# Initialize the cumulative heat load column with float type
data['CumulativeHeatLoad'] = 0.0  # Explicitly set as float

# Iterate through the data to calculate cumulative heat load correctly
for i in range(1, len(data)):
    previous_cumulative = data.at[i-1, 'CumulativeHeatLoad']
    current_heat_load = data.at[i, 'HeatLoad']
    if previous_cumulative + current_heat_load > 0:
        data.at[i, 'CumulativeHeatLoad'] = previous_cumulative + current_heat_load
    else:
        data.at[i, 'CumulativeHeatLoad'] = 0.0  # Ensure float is maintained

data.head(-5)

,FarmName_Pseudo,SE_Number,AnimalNumber,StartDate,StartTime,LactationNumber,DaysInMilk,TotalYield,DateTime,YearSeason,...,CullDecisionDate,Temperature,RelativeHumidity,THI_adj,HW,cum_HW,Temp15Threshold,Age,HeatLoad,CumulativeHeatLoad
0,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,06:25:00,7,191,13.90,2022-01-01 06:25:00,2022-1,...,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095,-32.987056,0.000000
1,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,16:41:00,7,191,16.87,2022-01-01 16:41:00,2022-1,...,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095,-32.987056,0.000000
2,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,15:29:00,7,192,20.41,2022-01-02 15:29:00,2022-1,...,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000
3,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,03:31:00,7,192,16.28,2022-01-02 03:31:00,2022-1,...,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000
4,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,22:44:00,7,192,11.53,2022-01-02 22:44:00,2022-1,...,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364930,f454e660,SE-fcdf259d-0044-0,1044,2023-06-07,00:51:00,10,351,5.63,2023-06-07 00:51:00,2023-3,...,NaT,15.645833,0.731917,61.559237,0,0,1,4154,0.559237,0.559237
1364931,f454e660,SE-fcdf259d-0044-0,1044,2023-06-07,11:17:00,10,351,3.34,2023-06-07 11:17:00,2023-3,...,NaT,15.645833,0.731917,61.559237,0,0,1,4154,0.559237,1.118475
1364932,f454e660,SE-fcdf259d-0044-0,1044,2023-06-08,17:01:00,10,352,6.96,2023-06-08 17:01:00,2023-3,...,NaT,15.570833,0.601708,59.383267,0,0,1,4155,-1.616733,0.000000
1364933,f454e660,SE-fcdf259d-0044-0,1044,2023-06-08,02:23:00,10,352,8.18,2023-06-08 02:23:00,2023-3,...,NaT,15.570833,0.601708,59.383267,0,0,1,4155,-1.616733,0.000000


In [11]:
# When CumulativeHeatLoad is greater than 5, it indicates that the cow is under heat stress
data['HeatStress'] = (data['CumulativeHeatLoad'] > 5).astype(int)
data.head(-5)

,FarmName_Pseudo,SE_Number,AnimalNumber,StartDate,StartTime,LactationNumber,DaysInMilk,TotalYield,DateTime,YearSeason,...,Temperature,RelativeHumidity,THI_adj,HW,cum_HW,Temp15Threshold,Age,HeatLoad,CumulativeHeatLoad,HeatStress
0,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,06:25:00,7,191,13.90,2022-01-01 06:25:00,2022-1,...,-3.025000,0.930917,28.012944,0,0,0,3095,-32.987056,0.000000,0
1,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,16:41:00,7,191,16.87,2022-01-01 16:41:00,2022-1,...,-3.025000,0.930917,28.012944,0,0,0,3095,-32.987056,0.000000,0
2,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,15:29:00,7,192,20.41,2022-01-02 15:29:00,2022-1,...,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000,0
3,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,03:31:00,7,192,16.28,2022-01-02 03:31:00,2022-1,...,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000,0
4,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,22:44:00,7,192,11.53,2022-01-02 22:44:00,2022-1,...,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364930,f454e660,SE-fcdf259d-0044-0,1044,2023-06-07,00:51:00,10,351,5.63,2023-06-07 00:51:00,2023-3,...,15.645833,0.731917,61.559237,0,0,1,4154,0.559237,0.559237,0
1364931,f454e660,SE-fcdf259d-0044-0,1044,2023-06-07,11:17:00,10,351,3.34,2023-06-07 11:17:00,2023-3,...,15.645833,0.731917,61.559237,0,0,1,4154,0.559237,1.118475,0
1364932,f454e660,SE-fcdf259d-0044-0,1044,2023-06-08,17:01:00,10,352,6.96,2023-06-08 17:01:00,2023-3,...,15.570833,0.601708,59.383267,0,0,1,4155,-1.616733,0.000000,0
1364933,f454e660,SE-fcdf259d-0044-0,1044,2023-06-08,02:23:00,10,352,8.18,2023-06-08 02:23:00,2023-3,...,15.570833,0.601708,59.383267,0,0,1,4155,-1.616733,0.000000,0


In [12]:
# Calculate the DailyYield for each cow each day
data['DailyYield'] = data.groupby(['SE_Number', 'StartDate'])['TotalYield'].transform('sum')

# Sort the data by AnimalNumber and StartDate
data.sort_values(['AnimalNumber', 'StartDate'], inplace=True)

# Calculate the previous day's total yield for each cow
data['PreviousDailyYield'] = data.groupby('AnimalNumber')['DailyYield'].shift(1)

# Calculate the daily yield change for each cow
data['DailyYieldChange'] = data['DailyYield'] - data['PreviousDailyYield']

# Group and aggregate data
data = data.groupby(['SE_Number', 'FarmName_Pseudo', 'StartDate']).agg({
    'DailyYield': 'first',
    'PreviousDailyYield': 'first',
    'DailyYieldChange': 'first',
    'HW': 'max',
    'Temperature': 'mean',
    'THI_adj': 'mean',
    'DaysInMilk': 'first',
    'YearSeason': 'first',
    'cum_HW': 'max',
    'Temp15Threshold': 'max',
    'Age': 'first',
    'BreedName': 'first',
    'LactationNumber': 'first',
    'HeatLoad': 'mean',
    'CumulativeHeatLoad': 'mean',
    'HeatStress': 'max'
}).reset_index()

# Renaming and formatting
data.rename(columns={
    'Temperature': 'MeanTemperature',
    'THI_adj': 'MeanTHI_adj',
    'StartDate': 'Date'
}, inplace=True)
data['Date'] = pd.to_datetime(data['Date'])

# Display the first few rows of the transformed data
data.head()

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,YearSeason,cum_HW,Temp15Threshold,Age,BreedName,LactationNumber,HeatLoad,CumulativeHeatLoad,HeatStress
0,SE-064c0cec-1189,a624fb9a,2022-01-01,30.77,30.77,0.00,0,-3.025000,28.012944,191,2022-1,0,0,3095,02 SLB,7,-32.987056,0.0,0
1,SE-064c0cec-1189,a624fb9a,2022-01-02,48.22,30.77,17.45,0,-0.279167,32.898193,192,2022-1,0,0,3096,02 SLB,7,-28.101807,0.0,0
2,SE-064c0cec-1189,a624fb9a,2022-01-03,30.53,48.22,-17.69,0,2.033333,36.760487,193,2022-1,0,0,3097,02 SLB,7,-24.239513,0.0,0
3,SE-064c0cec-1189,a624fb9a,2022-01-04,42.26,30.53,11.73,0,0.066667,31.939524,194,2022-1,0,0,3098,02 SLB,7,-29.060476,0.0,0
4,SE-064c0cec-1189,a624fb9a,2022-01-05,38.49,42.26,-3.77,0,-3.700000,26.498206,195,2022-1,0,0,3099,02 SLB,7,-34.501794,0.0,0


In [13]:
# Check if DailyYield is centered around approx the same for each farm
print("Mean of DailyYield:", data.groupby('FarmName_Pseudo')['DailyYield'].mean())
print("Standard Deviation of DailyYield:", data.groupby('FarmName_Pseudo')['DailyYield'].std())

Mean of DailyYield: FarmName_Pseudo
5c06d92d    37.389675
752efd72    31.151716
a624fb9a    33.413694
f454e660    30.485127
Name: DailyYield, dtype: float64
Standard Deviation of DailyYield: FarmName_Pseudo
5c06d92d     9.960240
752efd72     7.799288
a624fb9a    11.050811
f454e660    11.833056
Name: DailyYield, dtype: float64


In [16]:
# Define the Wilmink Lactation Curve function
def wilmink_lactation_curve(dim, a, b, c, d):
    dim = np.array(dim, dtype=float)
    return a + b * dim + c * np.exp(-d * dim)

# Function to detect and remove outliers
def remove_outliers(group, threshold=3.5):
    mean = np.mean(group['DailyYield'])
    std_dev = np.std(group['DailyYield'])
    return group[(group['DailyYield'] > mean - threshold * std_dev) & (group['DailyYield'] < mean + threshold * std_dev)]

# Function to smooth the data using a rolling average
def smooth_data(group, window=5):
    group = group.copy()
    group['DailyYield'] = group['DailyYield'].rolling(window, min_periods=1).mean()
    return group

# Simplified test quantile loss function
def quantile_loss(params, dim, yield_data, quantile=0.7):
    try:
        a, b, c, d = params
    except Exception as e:
        print(f"Error unpacking parameters: {e}")
        return np.inf  # Return a large loss if there's an error
    
    predictions = wilmink_lactation_curve(dim, a, b, c, d)
    residuals = yield_data - predictions
    
    # Ensure this returns a single scalar
    loss = np.sum(np.maximum(quantile * residuals, (quantile - 1) * residuals))
    
    print(f"Returning loss: {loss}")
    return loss

# Function to fit the Wilmink Lactation Curve to the dataset
def fit_wilmink_lactation_curve(dataset, quantile=0.7):
    dataset['ExpectedYield'] = np.nan
    params_dict = {}
    valid_indices = []

    for (animal_number, lactation_number), group in tqdm(dataset.groupby(['SE_Number', 'LactationNumber']), unit=" Segments"):
        print(f"Processing cow {animal_number}, lactation {lactation_number}")
        group = remove_outliers(group, threshold=3.5)
        group = smooth_data(group)
        x_data = group['DaysInMilk'].values
        y_data = group['DailyYield'].values

        print(f"x_data shape: {x_data.shape}, y_data shape: {y_data.shape}")

        if not np.isfinite(x_data).all() or not np.isfinite(y_data).all():
            print(f"Skipping due to non-finite values.")
            continue
        
        if len(x_data) < 10 or len(y_data) < 10:
            print(f"Skipping due to insufficient data points.")
            continue

        valid_indices.extend(group.index)

        try:
            initial_guesses = [np.mean(y_data), 0, np.mean(y_data) / 2, 0.1]
            bounds = ([-np.inf, -np.inf, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf])

            result = minimize(
                quantile_loss, initial_guesses, args=(x_data, y_data, quantile),
                method='L-BFGS-B', bounds=bounds, options={'maxiter': 30000}
            )

            print(f"Optimization result: {result}")
            
            if result.success:
                popt = result.x
                params_dict[(animal_number, lactation_number)] = {'a': popt[0], 'b': popt[1], 'c': popt[2], 'd': popt[3]}
                dataset.loc[group.index, 'ExpectedYield'] = wilmink_lactation_curve(group['DaysInMilk'], *popt)
            else:
                print(f"Optimization failed: {result.message}")
        except Exception as e:
            print(f"Optimization error: {e}")
            continue

    dataset = dataset.loc[valid_indices].reset_index(drop=True)
    dataset['ExpectedYield'] = dataset['ExpectedYield'].fillna(0)
    return dataset, params_dict

# Run the simplified fitting process
data, params_dict = fit_wilmink_lactation_curve(data, quantile=0.7)


data

  9%|▊         | 198/2315 [00:00<00:01, 1128.45 Segments/s]

Processing cow SE-064c0cec-1189, lactation 7
x_data shape: (88,), y_data shape: (88,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-064c0cec-1189, lactation 8
x_data shape: (207,), y_data shape: (207,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-30dc5787-1389, lactation 5
x_data shape: (34,), y_data shape: (34,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-30dc5787-1389, lactation 6
x_data shape: (327,), y_data shape: (327,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-30dc5787-1389, lactation 7
x_data shape: (171,), y_data shape: (171,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-30dc5787-1396, lactation 5
x_data shape: (194,), y_data shape: (194,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-30dc5787-1396, lactation 6
x_data shape: (348,), y_data shape: (348,)
Optimization error: too many value

 33%|███▎      | 756/2315 [00:00<00:00, 1690.19 Segments/s]

Processing cow SE-5c06d92d-3007, lactation 2
x_data shape: (208,), y_data shape: (208,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-5c06d92d-3007, lactation 3
x_data shape: (276,), y_data shape: (276,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-5c06d92d-3007, lactation 4
x_data shape: (78,), y_data shape: (78,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-5c06d92d-3008, lactation 2
x_data shape: (239,), y_data shape: (239,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-5c06d92d-3008, lactation 3
x_data shape: (256,), y_data shape: (256,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-5c06d92d-3009, lactation 2
x_data shape: (424,), y_data shape: (424,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-5c06d92d-3009, lactation 3
x_data shape: (183,), y_data shape: (183,)
Optimization error: too many val

 49%|████▊     | 1126/2315 [00:00<00:00, 1777.69 Segments/s]

...

 81%|████████  | 1866/2315 [00:01<00:00, 1741.90 Segments/s]

x_data shape: (190,), y_data shape: (190,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-752efd72-0494, lactation 1
x_data shape: (185,), y_data shape: (185,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-752efd72-0495, lactation 1
x_data shape: (149,), y_data shape: (149,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-752efd72-0496, lactation 1
x_data shape: (184,), y_data shape: (184,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-752efd72-0497, lactation 1
x_data shape: (17,), y_data shape: (17,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-752efd72-0498, lactation 1
x_data shape: (41,), y_data shape: (41,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-752efd72-0499, lactation 1
x_data shape: (128,), y_data shape: (128,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-75

100%|██████████| 2315/2315 [00:01<00:00, 1731.07 Segments/s]

x_data shape: (182,), y_data shape: (182,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-a624fb9a-1491, lactation 1
x_data shape: (184,), y_data shape: (184,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-a624fb9a-1493, lactation 1
x_data shape: (116,), y_data shape: (116,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-a624fb9a-1496, lactation 1
x_data shape: (138,), y_data shape: (138,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-a624fb9a-1500, lactation 1
x_data shape: (75,), y_data shape: (75,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-a624fb9a-1503, lactation 1
x_data shape: (123,), y_data shape: (123,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-a624fb9a-1505, lactation 1
x_data shape: (83,), y_data shape: (83,)
Optimization error: too many values to unpack (expected 2)
Processing cow SE-a6

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,YearSeason,cum_HW,Temp15Threshold,Age,BreedName,LactationNumber,HeatLoad,CumulativeHeatLoad,HeatStress,ExpectedYield
0,SE-064c0cec-1189,a624fb9a,2022-01-01,30.77,30.77,0.00,0,-3.025000,28.012944,191,2022-1,0,0,3095,02 SLB,7,-32.987056,0.000000,0,0.0
1,SE-064c0cec-1189,a624fb9a,2022-01-02,48.22,30.77,17.45,0,-0.279167,32.898193,192,2022-1,0,0,3096,02 SLB,7,-28.101807,0.000000,0,0.0
2,SE-064c0cec-1189,a624fb9a,2022-01-03,30.53,48.22,-17.69,0,2.033333,36.760487,193,2022-1,0,0,3097,02 SLB,7,-24.239513,0.000000,0,0.0
3,SE-064c0cec-1189,a624fb9a,2022-01-04,42.26,30.53,11.73,0,0.066667,31.939524,194,2022-1,0,0,3098,02 SLB,7,-29.060476,0.000000,0,0.0
4,SE-064c0cec-1189,a624fb9a,2022-01-05,38.49,42.26,-3.77,0,-3.700000,26.498206,195,2022-1,0,0,3099,02 SLB,7,-34.501794,0.000000,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483102,SE-fcdf259d-0044-0,f454e660,2023-06-08,15.14,8.97,6.17,0,15.570833,59.383267,352,2023-3,0,1,4155,41 Fjällko,10,-1.616733,0.000000,0,0.0
483103,SE-fcdf259d-0044-0,f454e660,2023-06-09,7.47,15.14,-7.67,0,13.254167,54.534255,353,2023-3,0,1,4156,41 Fjällko,10,-6.465745,0.000000,0,0.0
483104,SE-fcdf259d-0044-0,f454e660,2023-06-10,14.73,7.47,7.26,0,13.258333,54.082367,354,2023-3,0,1,4157,41 Fjällko,10,-6.917633,0.000000,0,0.0
483105,SE-fcdf259d-0044-0,f454e660,2023-06-12,12.27,14.73,-2.46,0,15.820833,62.015093,356,2023-3,0,1,4159,41 Fjällko,10,1.015093,1.015093,0,0.0
